<a href="https://colab.research.google.com/github/MrZombie69232/Simplilearn_Projects/blob/main/RAINBENCH_DATASET_EXPLORE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![spaceml](https://drive.google.com/uc?id=1fwi6BLJhnMv0vkWfdKbscgHvtPNEparz)
# RainBench Notebook 
In this notebook, we will demonstrate how to download and explore a small sample (1GB) of the main RainBench dataset.<br>
Our full data loaders and machine learning pipelines can be found at our public code repository (https://github.com/FrontierDevelopmentLab/PyRain.git). <br>
Results from the analysis of the main dataset is published at AAAI (https://arxiv.org/abs/2012.09670):<br>

##The Digital Twin Team<br>
Researchers:<br>

*Christian Schroeder de Witt - University of Oxford*<br>
*Catherine Tong - University of Oxford*<br>
*Valentina Zantedeschi - Inria, Lille and University College London*<br>
*Daniele De Martini - University of Oxford*<br>

Faculty:<br>

*Freddie Kalaitzi - University of Oxford*<br>
*Matthew Chantry - University of Oxford*<br>
*Duncan Watson-Parris - University of Oxford*<br>
*Piotr Bilinski - University of Oxford and University of Warsaw*<br>

**This notebook will let you explore the RainBench Dataset**


# Getting set up
In this section we will import required packages, get you authenticated and set up with the INARA public data bucket.

In [ ]:
import os
import numpy as np
import pandas as pd
from io import StringIO
import datetime
from google.cloud import storage
from google.colab import auth
import torch
import json

**The cell below will ask you to authenticate with a Google Cloud**

In [ ]:
#Authenticate user
auth.authenticate_user()



**Now we will connect to a GCP bucket and have a look at what's inside**

In [ ]:
bucket_name = 'aaai_release'
client = storage.Client(project='fdl-europe-dte')
gcs_bucket = client.get_bucket(bucket_name)

print('Current files in bucket:')
# List all the available files in bucket
file_list = list(gcs_bucket.list_blobs())
#print all files in the bucket as blobs
for f in file_list:
  print(f)

# Download Data

**We will only download and explore the data from the samples folder**



In [ ]:
# create directories to save data
!mkdir samples/
!mkdir samples/era5625_sample/
!mkdir samples/imerg5625_sample/
!mkdir samples/simsat5625_sample/

In [ ]:
# only download the files from samples folder (this takes about a minute)
dir = 'samples'
file_list = list(gcs_bucket.list_blobs(prefix=dir))
for blob in file_list:
  blob.download_to_filename(blob.name)

In [ ]:
datapath = ["/content/samples/era5625_sample/era5625_sample.dill", 
            "/content/samples/imerg5625_sample/imerg5625_sample.dill", 
            "/content/samples/simsat5625_sample/simsat5625_sample.dill"]

# Read Data

**We will now use the Dataset object from our code repository to parse the sampled data**

**We will need to supply the Datset with the following:**


1.   Dataset paths: specified by the *.dill* files
2.   Partition Configuration: how to partition the dataset temporally
3.   Sampling Configuration: what variables to sample when



In [ ]:
rootdir = os.getcwd()
datapath = ["samples/era5625_sample/era5625_sample.dill", 
            "samples/imerg5625_sample/imerg5625_sample.dill", 
            "samples/simsat5625_sample/simsat5625_sample.dill"]
datapath = [rootdir + '/' + p for p in datapath]

In [ ]:
# write partition configuration

# We specify the time ranges for train and test set respectively. 

partition_conf = {"train":
    {"timerange": (
        datetime.datetime(2018, 1, 1, 0).timestamp(), datetime.datetime(2018, 12, 31, 0).timestamp()),
        "increment_s": 60 * 60},
    "test":
        {"timerange": (datetime.datetime(2019, 1, 15, 0).timestamp(),
                       datetime.datetime(2019, 12, 31, 23).timestamp()),
         "increment_s": 60 * 60}}
partition_type = "range"

In [ ]:
# write sampling configurations

# "sample" can be interpreted as the input x to a Machine learning model, and "label" as its output y.
# Thus, under "sample", we specify the input variables. 
# Given as example are land-sea mask (lsm), latitude (lat), 2-metre temperature (t2m), surface pressure (sp).
# t2m and sp are temporal variables which we would like to sample from -3 to 0 hours from the event horizon.
# Next, under "label", we specify the output variable, precipitation (tp).
# We would like to sample this under 4 modes: +1 to +9 hours from the event horizon.

sample_conf = {"lead_time_{}".format(int(lt / 3600)):  # sample modes
    {
        "sample":  # sample sections
            {
                "lsm": {"vbl": "era5625/lsm"},  # sample variables
                "lat": {"vbl": "era5625/lat2d"},
                "t2m": {"vbl": "era5625/t2m",
                             "t": np.array([0, -1, -2, -3, ]) * 3600,
                             "interpolate": ["nan", "nearest_past", "nearest_future"][1]},
                "sp": {"vbl": "era5625/sp",
                             "t": np.array([0, -1, -2, -3, ]) * 3600,
                             "interpolate": ["nan", "nearest_past", "nearest_future"][1]},
            },
        "label":
            {
                "tp": {"vbl": "era5625/tp",
                       "t": np.array([lt]),
                       "interpolate": ["nan", "nearest_past", "nearest_future"][1]}}
    }
    for lt in np.array([1, 3, 6, 9]) * 3600}

**We import the Dataset object from our repository**

In [ ]:
!git clone https://github.com/FrontierDevelopmentLab/PyRain.git

In [ ]:
cd PyRain/

In [ ]:
from src.dataloader import Dataset

In [ ]:
# load dataset using defined configurations
dataset = Dataset(datapath=datapath,
                  partition_conf=partition_conf,
                  partition_type=partition_type,
                  partition_selected="train",
                  sample_conf=sample_conf,
                  )

In [ ]:
# One may inspect the corresponding .json file, which details the precise variable name contained in the memmap files (e.g. era5625/lat2d).
with open('/content/samples/era5625_sample/era5625_sample_info.json', 'rb') as f:
  json_file = json.load(f)


# Exploring the Dataset



**One can now use a dataloader object to read batches of data from the dataset.**

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset, batch_size=2, shuffle=True)

We can inspect an example batch and find that each datapoint contains the following:



1.  samples
2.  labels
3.   sample modes
4.  sample timestamps



In [ ]:
batch = next(iter(loader))

In [ ]:
print(batch[0].keys())

dict_keys(['sample', 'label', '__sample_modes__', '__sample_ts__'])


In [ ]:
# We can inspect a random datapoint from the batch.

idx = 1

print('Sample Mode:', batch[0]['__sample_modes__'][idx])
print('\nSample timestamps:')

for ts in batch[0]['sample']['t2m__ts'][idx]:
  print(datetime.datetime.fromtimestamp(ts))

print('\nLast time in sample:')
print(datetime.datetime.fromtimestamp(batch[0]['__sample_ts__'][0][idx]))

print('\nLabel timestamps:')
print(datetime.datetime.fromtimestamp(batch[0]['label']['tp__ts'][idx,0]))

Sample Mode: lead_time_3

Sample timestamps:
2018-03-04 13:00:00
2018-03-04 12:00:00
2018-03-04 11:00:00
2018-03-04 10:00:00

Last time in sample:
2018-03-04 13:00:00

Label timestamps:
2018-03-04 16:00:00


We can also inspect the shapes of the data returned

In [ ]:
print('lat: ', batch[0]['sample']['lat'].shape)
print('t2m: ', batch[0]['sample']['t2m'].shape)

lat:  torch.Size([2, 1, 32, 64])
t2m:  torch.Size([2, 4, 1, 32, 64])


The dataset also supports parsing by timestamps and data categories.

In [ ]:
tp = dataset[((datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp(), 3600), ["era5625/tp"], None)]
imerg = dataset[((datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp(), 3600), ["imerg5625/precipitationcal"], None)]
simsat = dataset[((datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp(), 3*3600), ["simsat5625/clbt:0"], {"interpolate":"nearest_past"})]
simsat2 = dataset[([datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp()], ["simsat5625/clbt:0"], {})]
tp = dataset[((datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp(), 3600), ["era5625/sp"], None)]
tp = dataset[((datetime.datetime(2018,1,1,0).timestamp(), datetime.datetime(2018,12,31,0).timestamp(), 3600), ["era5625/t2m"], None)]